In [ ]:
# basic imports
import pandas as pd

# preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# vectorization
from sklearn.feature_extraction.text import CountVectorizer

# topic model
from bertopic import BERTopic

# Data Loading

In [5]:
df = pd.read_csv("Dataset/oncology.csv",
                 sep = ';') # separator
df.head()

,cancer,Comment,tokens,stopwordremove_tokens,lemmatized_text,none
0,breast cancer,tiredness got better edema ankle several year ...,"['tiredness', 'that', 'got', 'better', 'edema'...","['tiredness', 'got', 'better', 'edema', 'ankle...","['tiredness', 'got', 'better', 'edema', 'ankle...",NaN
1,none,one people could take attack body along cancer...,"['one', 'people', 'could', 'take', 'attack', '...","['one', 'people', 'could', 'take', 'attack', '...","['one', 'people', 'could', 'take', 'attack', '...",NaN
2,none,take month week mg lymphedema arm get infect a...,"['take', 'month', 'week', 'mg', 'lymphedema', ...","['take', 'month', 'week', 'mg', 'lymphedema', ...","['take', 'month', 'week', 'mg', 'lymphedema', ...",NaN
3,none,stage lymph bone bone metastasis never infusio...,"['stage', 'lymph', 'bone', 'bone', 'metastasis...","['stage', 'lymph', 'bone', 'bone', 'metastasis...","['stage', 'lymph', 'bone', 'bone', 'metastasis...",NaN
4,none,although medication effectively treat cancer s...,"['although', 'medication', 'effectively', 'tre...","['although', 'medication', 'effectively', 'tre...","['although', 'medication', 'effectively', 'tre...",NaN


In [6]:
# dimensions
df.shape

(14419, 6)

In [7]:
# only extract patient messages
msgs = df['Comment'].tolist()
msgs[0]

'tiredness got better edema ankle several year hair thinningjoint pain year afinitor doctor said consider going drug cause high blood sugar high cholesterol neither opted go month monthly hormone injection cancer still stable'

# Preprocessing

In [ ]:
# tokenize


#  


In [ ]:
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(msgs)


# Topic Modeling

Chosen Topic Model: BERTopic

In [9]:
# 